# 说明

很多经典的网络，都已经在ImageNet数据集（1000类，海量训练集和测试集）上进行过最好的训练了，获得了对应的最好的权重系数！—— 卷积层的作用：最主要的功能就是特征提取。所以，即使最后分类的目标不同，但优秀的卷积特征提取部分还是通用的！

因此，使用预训练网络，尤其是还使用其非全连接层的所有已训练好的权重系数，可以极大提高自己的图像分类任务的同时，极大降低自己网络的训练参数（因为最多的参数都集中在卷积部分，而这些不需要我们训练）。

预训练网络 = 迁移学习 —— 权值共享 + 网络微调

两种keras高阶模块：
- tf.keras.applications.xxx：使用各种预训练网络
- tf.keras.datasets.xxx：使用各种常见数据集

**迁移学习操作**：
- 导入预训练网络，并使用已训练好的“**卷积基部分**”的权重参数；
- 然后丢弃掉原始网络的头部（高阶特征提取部分），换成适合自己任务的头部Dense层；
- 最后冻结新的网络中卷积层部分的所有参数不参与训练！只训练自己添加的头部Dense层参数。

---
微调预训练网络：

首先要明确：
- 网络的**底层/尾部**：指靠近“**输入层**”的那部分（卷积层），主要负责提取**广泛/通用**、低阶的特征；
- 网络的**顶层/头部**：指靠近“**输出层**”的那部分（Dense层），主要负责提取**抽象/特定**、高阶的特征；—— 迁移学习 + 微调

**微调的操作**：一定是迁移学习完成后（自己的Dense层已经训练好），才继续进行微调！
- 解冻卷积基靠上的一部分层（靠近自定的Dense层的3-5个卷积层）；
- 再次联合训练解冻的卷积层和自定义的Dense层！—— 使上层卷积基（提取特定高阶任务）更加**配套于**自定的Dense层/任务！

注意：
- 为什么要先完成一次自定的Dense层的训练，而不是一上来就用微调的思路来进行？ —— 因为：若Dense从未经过任何训练，那它的初始精度是非常差的！微调的核心思路就是在已经不错的网络上，再稍微提高一点，而不是一上来就要一步到位！
- 微调时的学习速度一定要更小更慢！因为之前已经有很好的结果说明模型已经靠近全局最优了！故继续微调优化的速率一定要足够慢！
- 前后两次的history记录不要用同名，否则后者微调时会把之前的history记录替换掉！！

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import glob
import os

In [2]:
# 所有文件的地址：
train_image_path = glob.glob('E:/tensorflow2.0_日月光华/日月光华-tensorflow资料/数据集/dc_2000/train/*/*.jpg')
test_image_path = glob.glob('E:/tensorflow2.0_日月光华/日月光华-tensorflow资料/数据集/dc_2000/test/*/*.jpg')

In [3]:
# 标签提取：
train_image_label = [ int(p.split('\\')[1] == 'cat') for p in train_image_path]
test_image_label = [ int(p.split('\\')[1] == 'cat') for p in test_image_path]

In [4]:
# 每个batch训练：
def pre_processing_train_image(path, label):
    
    # 对训练图像的预处理：
    image = tf.io.read_file( path )                       # 文件读取
    image = tf.image.decode_jpeg( image, channels = 3 )   # 文件解码成jpg图片，并给定图片的通道数（默认为0！）
    image = tf.image.resize( image, [300,300] )           # 等比缩放，不要选裁剪！
    image = tf.cast( image, tf.float32 )                  # 转换数据类型：读入默认是int8
    image = image / 255                                   # 归一化
    
    # 图像增强：前3个最常用！
    image = tf.image.random_crop(image, [256,256,3])  # 随机裁剪：必须是3通道
    image = tf.image.random_flip_left_right(image)    # 随机左右翻转
    image = tf.image.random_flip_up_down(image)       # 随机上下翻转
    # image = tf.image.random_brightness(image, 0.5)    # 随机亮度
    # image = tf.image.random_contrast(image, 0, 1)     # 随机对比度
    
    # 对标签的预处理：每张图作为一个独立的数据，其标签已经在一个列表中，总体形成嵌套列表：[1,1,0] -- [ [1], [1], [0] ]
    label = tf.reshape(label, [1])
        
    return image, label

In [5]:
# 每个batch测试：
def pre_processing_test_image(path, label):
    
    # 对训练图像的预处理：
    image = tf.io.read_file( path )                       # 文件读取
    image = tf.image.decode_jpeg( image, channels = 3 )   # 文件解码成jpg图片，并给定图片的通道数（默认为0！）
    image = tf.image.resize( image, [256,256] )           # 等比缩放，不要选裁剪！
    image = tf.cast( image, tf.float32 )                  # 转换数据类型：读入默认是int8
    image = image / 255                                   # 归一化
    
    # 对标签的预处理：每张图作为一个独立的数据，其标签已经在一个列表中，总体形成嵌套列表：[1,1,0] -- [ [1], [1], [0] ]
    label = tf.reshape(label, [1])
        
    return image, label

In [6]:
# 用tf.data创建输入数据集：
AUTOTUNE = tf.data.experimental.AUTOTUNE   # 新操作：在tf.data模块使用时，会自动根据cpu来情况进行并行计算处理！

# 训练数据：
train_dataset = tf.data.Dataset.from_tensor_slices( (train_image_path, train_image_label) )
train_dataset = train_dataset.map( pre_processing_train_image, num_parallel_calls = AUTOTUNE )  # 这里用到上面预处理的函数
# 测试数据：
test_dataset = tf.data.Dataset.from_tensor_slices( (test_image_path, test_image_label) )
test_dataset = test_dataset.map( pre_processing_test_image, num_parallel_calls = AUTOTUNE )  # 这里用到上面预处理的函数

In [7]:
# 乱序、批划分：
BATCH_SIZE = 16
# 训练数据：
train_count = len( train_image_path )  # 总数据个数
train_dataset = train_dataset.shuffle(train_count).repeat().batch(BATCH_SIZE)  # 乱序 + 划分批次 
train_dataset = train_dataset.prefetch(AUTOTUNE)  # 新操作：预取到缓存：加速处理
# 测试数据：
test_dataset = test_dataset.batch(BATCH_SIZE)  # 乱序 + 划分批次 
test_dataset = test_dataset.prefetch(AUTOTUNE)  # 新操作：预取到缓存：加速处理

### 网络搭建：使用预训练网络

In [8]:
# 权重是关于imagenet数据集训练得来的；不包含头部的全连接层
# weights需要下载，当然可以下载好的文件（.h5）放在C盘用户文件夹里的.keras/models里
conv_base = tf.keras.applications.VGG16( weights = 'imagenet', include_top = False ) 

In [9]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

模型创建开始：

In [11]:
model = tf.keras.Sequential([
    conv_base,  # 就是一个层！但这一层参数不希望训练！
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 14,977,857
Non-trainable params: 0
_________________________________________________________________


In [13]:
conv_base.trainable = False  # 该层不可训练

In [14]:
model.summary()  # 需训练参数大减！

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,977,857
Trainable params: 263,169
Non-trainable params: 14,714,688
_________________________________________________________________


直接用tf.keras高阶完成编译和训练了：

In [15]:
model.compile(
    optimizer = tf.keras.optimizers.Adam( lr = 0.0005 ),
    loss = 'binary_crossentropy',
    metrics = ['acc']
)

In [ ]:
epoch_num = 15

history1 = model.fit(
    train_dataset,
    epochs = epoch_num,
    steps_per_epoch = len(train_image_path) // BATCH_SIZE,
    validation_data = test_dataset,
    validation_steps = len(test_image_path) // BATCH_SIZE
)

### 微调预训练网络：

In [16]:
# 解冻头部部分卷积层：
conv_base.trainable = True    # 先全解冻

In [18]:
len(conv_base.layers)  # 查看一共有多少层！

19

In [19]:
# 设定只解冻最后的3层：
final_tune_at = -3

In [20]:
# 关闭最后3层之前的所有层：
for layer in conv_base.layers[:final_tune_at]:
    layer.trainable = False  

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 10,258,241
Trainable params: 263,169
Non-trainable params: 9,995,072
_________________________________________________________________


In [22]:
# 模型重新编译：
model.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam( lr = 0.0005 / 10 ),  # 微调：肯定是要更慢的下探！
    metrics = ['acc']
)

In [ ]:
# 新的训练：
initial_epochs = epoch_num
final_epochs = 10  # 再多训练10个epoch
total_epochs = initial_epochs + final_epochs

history2 = model.fit(
    train_dataset,
    steps_per_epoch = len(train_image_path) // BATCH_SIZE,
    epochs = total_epochs,  # 数值从initial_epochs开始记录而已！
    initial_epoch = initial_epochs,
    
    validation_data = test_dataset,
    validation_steps = len(test_image_path) // BATCH_SIZE
)